In [1]:
# IP adress of this computer.
HOST = "localhost"
# Port on which the server will listen.
PORT = 1300
# Whether to preview the camera pictures.
SHOW_VIDEO = True
# ID of camera to use.
CAMERA_ID = 1

In [3]:
import json
import socketserver
import os
import base64
from io import BytesIO
from PIL import Image
import cv2
import numpy as np

In [4]:
cam = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow("Preview")
cv2.namedWindow("Mask")

_, image = cam.read()
#print(image.shape[:2])
mask = np.ones(image.shape)
#print(mask.shape)

threshold = [0, 0, 0]
color_picked = np.zeros((3))

def on_change_threshold_h(value):
    threshold[0] = value
def on_change_threshold_s(value):
    threshold[1] = value
def on_change_threshold_v(value):
    threshold[2] = value

cv2.createTrackbar("threshold_h", "Preview", 0, 255, on_change_threshold_h)
cv2.createTrackbar("threshold_s", "Preview", 0, 255, on_change_threshold_s)
cv2.createTrackbar("threshold_v", "Preview", 0, 255, on_change_threshold_v)

def pickColor(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        _, image = cam.read()
        global color_picked
        color_picked = cv2.cvtColor(np.uint8([[image[y, x]]]), cv2.COLOR_BGR2HSV)[0][0]
        
        #image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        #global mask
        #mask = np.bitwise_and(mask, cv2.inRange(image_hsv, color_bound1, color_bound2).astype(dtype=bool))

cv2.setMouseCallback("Preview", pickColor)

In [5]:
while(SHOW_VIDEO):
    _, image = cam.read()
    
    image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    global color_picked
    color_bound1 = color_picked - threshold
    color_bound2 = color_picked + threshold
    mask = cv2.inRange(image_hsv, color_bound1, color_bound2)
    
    cv2.imshow("Preview", image)
    cv2.imshow("Mask", mask)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

In [6]:
threshold = 120
cv2.namedWindow("Result")

def on_change_threshold(value):
    global threshold
    threshold = value

cv2.createTrackbar("threshold", "Result", 0, 255, on_change_threshold)
        
while(True):
    _, inv_image = cam.read()
    inv_masked_image = cv2.bitwise_and(inv_image, inv_image, mask=mask)
    masked_image = cv2.cvtColor(np.invert(inv_masked_image), cv2.COLOR_BGR2GRAY)
    masked_image = cv2.threshold(masked_image,threshold,255,cv2.THRESH_BINARY)[1]
    #print(threshold)
    cv2.imshow("Result", masked_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break

In [ ]:
class TCPHandler(socketserver.StreamRequestHandler):

    def handle(self):
        print("client connected.")
        try:
            cam = cv2.VideoCapture(CAMERA_ID)
            _, raw_image = cam.read()
            inv_masked_image = cv2.bitwise_and(raw_image, raw_image, mask=mask)
            header = json.loads(self.rfile.readline().strip())["header"]
            if (header == "image"):
                _, image_arr = cv2.imencode('.jpg', inv_masked_image)
                image_bytes = image_arr.tobytes()
                image_b64 = base64.b64encode(image_bytes)
                self.wfile.write(image_b64)
            elif (header == "raw"):
                _, image_arr = cv2.imencode('.jpg', raw_image)
                image_bytes = image_arr.tobytes()
                image_b64 = base64.b64encode(image_bytes)
                self.wfile.write(image_b64)
            else:
                masked_image = cv2.cvtColor(np.invert(inv_masked_image), cv2.COLOR_BGR2GRAY)
                masked_image = cv2.threshold(masked_image,threshold,255,cv2.THRESH_BINARY)[1]
                _, image_arr = cv2.imencode('.jpg', masked_image)
                image_bytes = image_arr.tobytes()
                image_b64 = base64.b64encode(image_bytes)
                
                self.wfile.write(image_b64)
        except Exception as e:
            print(e.message, e.args)

with socketserver.TCPServer((HOST, PORT), TCPHandler) as server:
    print("Starting server.")
    server.serve_forever()

Starting server.
